In [1]:
import json
import os
import ast
import re
import pandas as pd

In [14]:
qat_results = [result for result in os.listdir("experiment_results") if 'qat_independent_YuNET' in result and "q_at" not in result and "eval_only" not in result]

In [15]:
print(qat_results)

['qat_independent_YuNET_adaround_rescaled_seed2', 'qat_independent_YuNET_cle_adaround_rescaled_seed0', 'qat_independent_YuNET_bnf_adaround_rescaled_seed0', 'qat_independent_YuNET_bnf_adaround_rescaled_seed2', 'qat_independent_YuNET_rescaled_seed1', 'qat_independent_YuNET_rescaled_seed0', 'qat_independent_YuNET_cle_adaround_rescaled_seed1', 'qat_independent_YuNET_bnf_cle_adaround_rescaled_seed1', 'qat_independent_YuNET_bnf_cle_adaround_rescaled_seed0', 'qat_independent_YuNET_bnf_cle_rescaled_seed2', 'qat_independent_YuNET_bnf_cle_rescaled_seed1', 'qat_independent_YuNET_bnf_cle_rescaled_seed0', 'qat_independent_YuNET_bnf_rescaled_seed0', 'qat_independent_YuNET_rescaled_seed2', 'qat_independent_YuNET_cle_rescaled_seed0', 'qat_independent_YuNET_adaround_rescaled_seed0', 'qat_independent_YuNET_bnf_cle_adaround_rescaled_seed2', 'qat_independent_YuNET_bnf_adaround_rescaled_seed1', 'qat_independent_YuNET_bnf_rescaled_seed2', 'qat_independent_YuNET_bnf_rescaled_seed1', 'qat_independent_YuNET_cl

In [10]:
pattern = r"step (\d+): ({[^}]+})"

def open_results_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            stripped_lines = [line.strip() for line in lines]
    except FileNotFoundError:
        print("The specified file was not found.")
    
    return stripped_lines

In [11]:
def generate_results_df(experiment_name = "pruning_pipelined_yunet", prefix = "after channel prunning"):
    result_dict_list = []
    for seed in range(3):
        file_path = f'results/{experiment_name}_seed{seed}.txt'
        result_list = open_results_file(file_path)
        for line in result_list:
            result_dict = {}
            if prefix in line:
                substring_match = re.search(pattern, line)
                if substring_match:
                    step_number = int(substring_match.group(1))
                    substring = substring_match.group(2)
                    # Evaluate the substring into a Python dictionary
                    try:
                        print(experiment_name)
                        if 'yunet' in experiment_name:
                            result_dict = ast.literal_eval(substring)
                            result_dict[f'NME'] = result_dict['NME'][0]
                            result_dict[f'IoU'] = result_dict['IoU'][0]
                            result_dict[f'seed'] = seed


                            result_dict.pop("n_detections")
                        else:
                            result_dict = ast.literal_eval(substring)
                            result_dict[f'seed'] = seed

                        result_dict['step'] = int(step_number)  # Add the 'step' key to the dictionary



                        result_dict_list.append(result_dict)
                    except SyntaxError:
                        print("Invalid syntax in the substring.")

                else:
                    print("No matching substring found.")
    result_df = pd.DataFrame(result_dict_list)
    return result_df

In [17]:

dict_for_df = {}
for result in sorted(qat_results):
    result_json_path = os.path.join(
        os.path.join("experiment_results",result),
        "qat_eval_results.json")
    if os.path.exists(result_json_path):
        print(result)
        with open(result_json_path, 'r') as json_file:
            data = json.load(json_file)
            print(data.keys())
            dict_for_df[result] = data['after QAT']
            dict_for_df[result]['NME'] = dict_for_df[result]['NME'][0]
            dict_for_df[result]['IoU'] = dict_for_df[result]['IoU'][0]

results_df = pd.DataFrame(dict_for_df)

display(results_df.T)
grouped_columns = results_df.groupby(lambda x: '_'.join(x.split('_')[:-1]), axis=1)
for group, columns in grouped_columns:

    results_df[f'{group}_average'] = columns.mean(axis=1)
    results_df[f'{group}_std'] = columns.std(axis=1)
    results_df[f'{group}_min'] = columns.min(axis=1)

average_columns = results_df.filter(like='_average')
std_columns = results_df.filter(like='_std')
min_columns = results_df.filter(like='_min')

display(average_columns)
display(std_columns)
display(min_columns)

qat_independent_YuNET_adaround_rescaled_seed0
qat_independent_YuNET_adaround_rescaled_seed1
qat_independent_YuNET_adaround_rescaled_seed2
qat_independent_YuNET_bnf_adaround_rescaled_seed0
qat_independent_YuNET_bnf_adaround_rescaled_seed1
qat_independent_YuNET_bnf_adaround_rescaled_seed2
qat_independent_YuNET_bnf_cle_adaround_rescaled_seed0
qat_independent_YuNET_bnf_cle_adaround_rescaled_seed1
qat_independent_YuNET_bnf_cle_adaround_rescaled_seed2
qat_independent_YuNET_bnf_cle_rescaled_seed0
qat_independent_YuNET_bnf_cle_rescaled_seed1
qat_independent_YuNET_bnf_cle_rescaled_seed2
qat_independent_YuNET_bnf_rescaled_seed0
qat_independent_YuNET_bnf_rescaled_seed1
qat_independent_YuNET_bnf_rescaled_seed2
qat_independent_YuNET_cle_adaround_rescaled_seed0
qat_independent_YuNET_cle_adaround_rescaled_seed1
qat_independent_YuNET_cle_adaround_rescaled_seed2
qat_independent_YuNET_cle_rescaled_seed0
qat_independent_YuNET_cle_rescaled_seed1
qat_independent_YuNET_cle_rescaled_seed2
qat_independent_YuN

""


C:\Users\jpradovieira\AppData\Local\Temp\ipykernel_17832\1007416867.py:19: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_columns = results_df.groupby(lambda x: '_'.join(x.split('_')[:-1]), axis=1)


""


""


""


In [13]:
generate_results_df("PipelinedYuNET", prefix="EVAL RESULT")

The specified file was not found.


UnboundLocalError: cannot access local variable 'stripped_lines' where it is not associated with a value

In [61]:
data_list = []
for results_folder in qat_results:
    json_file_path = os.path.join("experiment_results",os.path.join(results_folder, 'eval_results.json'))
    if os.path.exists(json_file_path):
        with open(json_file_path, 'r') as file:
            data = json.load(file)
            data['after pipelining']['NME'] = data['after pipelining']['NME'][0]
            data['after pipelining']['IoU'] = data['after pipelining']['IoU'][0]
            data_list.append(data['after pipelining'])

# Create a pandas DataFrame from the collected data
df = pd.DataFrame(data_list)
df.index = qat_results
# Display the resulting DataFrame

display(df)
grouped_columns = df.T.groupby(lambda x: '_'.join(x.split('_')[:-1]), axis=1)
for group, columns in grouped_columns:
    results_df[f'{group}_average'] = columns.mean(axis=1)
    results_df[f'{group}_std'] = columns.std(axis=1)
    results_df[f'{group}_min'] = columns.min(axis=1)

average_columns = results_df.filter(like='_average')
std_columns = results_df.filter(like='_std')
min_columns = results_df.filter(like='_min')

display(average_columns)
display(std_columns)
display(min_columns)

,NME,IoU,missed_detections,n_detections
ptq_independent_YuNET_bnf_rescaled_seed2,0.758898,0.689834,81,460
ptq_independent_YuNET_bnf_cle_adaround_rescaled_seed1,0.819103,0.697048,34,507
ptq_independent_YuNET_bnf_cle_rescaled_seed0,0.778800,0.700472,44,497
ptq_independent_YuNET_bnf_adaround_rescaled_seed0,0.834292,0.698729,47,494
ptq_independent_YuNET_adaround_rescaled_seed2,0.794429,0.710344,69,472
ptq_independent_YuNET_bnf_adaround_rescaled_seed1,0.826380,0.703183,41,500
ptq_independent_YuNET_adaround_rescaled_seed1,0.716672,0.705551,55,486
ptq_independent_YuNET_cle_rescaled_seed2,0.977676,0.681735,85,456
ptq_independent_YuNET_cle_rescaled_seed0,0.771740,0.700449,41,500
ptq_independent_YuNET_rescaled_seed2,0.797621,0.686845,82,459


C:\Users\jpradovieira\AppData\Local\Temp\ipykernel_22100\1504205869.py:17: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_columns = df.T.groupby(lambda x: '_'.join(x.split('_')[:-1]), axis=1)


,ptq_independent_YuNET_adaround_rescaled_average,ptq_independent_YuNET_bnf_adaround_rescaled_average,ptq_independent_YuNET_bnf_cle_adaround_rescaled_average,ptq_independent_YuNET_bnf_cle_rescaled_average,ptq_independent_YuNET_bnf_rescaled_average,ptq_independent_YuNET_cle_adaround_rescaled_average,ptq_independent_YuNET_cle_rescaled_average,ptq_independent_YuNET_rescaled_average
NME,0.750012,0.83576,0.758816,0.812054,0.845481,0.778721,0.840385,0.874373
IoU,0.707351,0.70100,0.703221,0.696407,0.695136,0.696584,0.694211,0.692417
missed_detections,59.000000,51.00000,46.666667,58.666667,57.666667,58.333333,55.666667,56.666667
n_detections,482.000000,490.00000,494.333333,482.333333,483.333333,482.666667,485.333333,484.333333


,ptq_independent_YuNET_adaround_rescaled_std,ptq_independent_YuNET_bnf_adaround_rescaled_std,ptq_independent_YuNET_bnf_cle_adaround_rescaled_std,ptq_independent_YuNET_bnf_cle_rescaled_std,ptq_independent_YuNET_bnf_rescaled_std,ptq_independent_YuNET_cle_adaround_rescaled_std,ptq_independent_YuNET_cle_rescaled_std,ptq_independent_YuNET_rescaled_std
NME,0.040044,0.010194,0.081441,0.057598,0.074984,0.049746,0.118897,0.066469
IoU,0.002609,0.002228,0.005762,0.007039,0.004591,0.007491,0.010805,0.004826
missed_detections,8.717798,12.489996,19.399313,25.403412,20.207259,18.929694,25.403412,21.939310
n_detections,8.717798,12.489996,19.399313,25.403412,20.207259,18.929694,25.403412,21.939310


,ptq_independent_YuNET_adaround_rescaled_min,ptq_independent_YuNET_bnf_adaround_rescaled_min,ptq_independent_YuNET_bnf_cle_adaround_rescaled_min,ptq_independent_YuNET_bnf_cle_rescaled_min,ptq_independent_YuNET_bnf_rescaled_min,ptq_independent_YuNET_cle_adaround_rescaled_min,ptq_independent_YuNET_cle_rescaled_min,ptq_independent_YuNET_rescaled_min
NME,0.716672,0.826380,0.666169,0.778800,0.758898,0.741241,0.771740,0.797621
IoU,0.705551,0.698729,0.697048,0.688279,0.689834,0.688302,0.681735,0.686845
missed_detections,53.000000,41.000000,34.000000,44.000000,46.000000,45.000000,41.000000,44.000000
n_detections,472.000000,476.000000,472.000000,453.000000,460.000000,461.000000,456.000000,459.000000


: 